In [1]:
from selenium import webdriver
import time, os

KLMS_URL = "http://klms.kaist.ac.kr/local/ubmessage/email_send.php?id=118949"
KLMS_LOG = "http://klms.kaist.ac.kr/local/ubmessage/email_log.php?id=118949"
KLMS_ID  = None
KLMS_PW  = None
FILE_DIR = "C:\\Users\\SeongJin\\Downloads\\cs459_practice0_graded"

SUBJECT = "CS459 - Automation Test"
CONTENT = """Dear {},

This is a test e-mail of CS459 automation.

If you received this e-mail, probably you are a TA.
If you are not a TA and got this e-mail, please contact seiker@kaist.ac.kr as soon as possible.

Best regards,
Sanghoon."""

In [2]:
browser = webdriver.Chrome("./chromedriver.exe")

browser.get("https://klms.kaist.ac.kr/login.php")

sso_btn = browser.find_element_by_css_selector("#login-form1 > a.btn.sso-login-btn")
sso_btn.click()

In [3]:
sso_form = browser.find_element_by_css_selector("body > div > div > div:nth-child(3) > div > div > fieldset > ul")

i = 0
for li in sso_form.find_elements_by_tag_name("li"):
    sso_input = li.find_element_by_tag_name("input")

    if i == 0:
        sso_input.send_keys(KLMS_ID)
    elif i == 1:
        sso_input.send_keys(KLMS_PW)
    elif i == 2:
        sso_input.click()

    i += 1

# KAIST SSO takes some time to be finished...
time.sleep(5)

In [4]:
for fname in os.listdir(FILE_DIR):
    klms_name = fname.split('_')[0]

    browser.get(KLMS_URL)
    browser.implicitly_wait(5)
    
    users_list = browser.find_elements_by_xpath("/html/body/div[2]/div[3]/div[2]/div/div/div[1]/div/div[2]/div/div/form/fieldset/div[1]/div/ul/li")

    # Processing checkbox
    users_dict = {}
    for li in users_list:
        user_label = li.find_element_by_tag_name("label")
        user_title = user_label.get_attribute("title")

        users_dict[user_title.split(" (")[0]] = user_label

    try:
        user_label = users_dict[klms_name]
        user_label.click()

        subject_field = browser.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div/div/div[1]/div/div[2]/div/div/form/fieldset/div[2]/div/input")
        content_field = browser.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div/div/div[1]/div/div[2]/div/div/form/fieldset/div[3]/div/div/div[1]/div[1]/div[2]/div")

        subject_field.send_keys(SUBJECT)
        content_field.send_keys(CONTENT.format(klms_name))

        file_btn = browser.find_element_by_xpath("/html/body/div[2]/div[3]/div[2]/div/div/div[1]/div/div[2]/div/div/form/fieldset/div[4]/div/div/div[2]/div[1]/div[1]/div[1]/a/img")
        file_btn.click()

        file_input = browser.find_element_by_xpath("/html/body/div[8]/div[2]/div/div[2]/div/div[2]/div[2]/div/div/form/div/div[1]/div/input")
        file_input.send_keys(FILE_DIR + '\\' + fname)

        file_upload_btn = browser.find_element_by_xpath("/html/body/div[8]/div[2]/div/div[2]/div/div[2]/div[2]/div/div/div/button")
        file_upload_btn.click()

        # Requires short sleep until the file is uploaded.
        time.sleep(2)

        send_btn = browser.find_element_by_xpath("/html/body/div[3]/div[3]/div[2]/div/div/div[1]/div/div[2]/div/div/form/fieldset/div[5]/button")
        send_btn.click()

        # Requires short sleep until the e-mail is sent.
        time.sleep(5)

        while (browser.current_url != "http://klms.kaist.ac.kr/local/ubmessage/email_log.php?id=118949"):
            time.sleep(1)

    except Exception as e:
        print("{} - error occurred.".format(klms_name))
        print(e)

In [ ]:
browser.current_url